## **Used Stored Procedures**

In [16]:
--TO ANSWER QUESTION NUMBER 1, 2, 2_1
CREATE OR ALTER PROC GET_CORR_DEMO_CHURN__ (@attr VARCHAR(20))
AS
BEGIN
      DECLARE @sql NVARCHAR(MAX);
    SET @sql = N'
    SELECT ' + QUOTENAME(@attr) + ',
           COUNT(CASE WHEN s.Customer_Status = ''Churned'' THEN ' + QUOTENAME(@attr) + ' END) AS Churned,
           COUNT(CASE WHEN s.Customer_Status = ''Stayed'' THEN ' + QUOTENAME(@attr) + ' END) AS Stayed
    FROM Demographics d
    JOIN Status s ON d.CustomerID = s.CustomerID
    GROUP BY ' + QUOTENAME(@attr) + ';';

    EXEC sp_executesql @sql;
	RETURN;
END

Commands completed successfully.

Total execution time: 00:00:00.090

In [9]:
--TO ANSWER QUESTION NMBER 6, 7, 14
CREATE OR ALTER PROCEDURE Average_OF_Column_And_GroupBy 
    @FirstCol NVARCHAR(20), 
    @AvgColumn NVARCHAR(20), 
    @GroupByColumn NVARCHAR(20), 
    @tableName NVARCHAR(20)
AS
BEGIN
    DECLARE @SqlQuery NVARCHAR(MAX);

    SET @SqlQuery = 
        'SELECT ' + QUOTENAME(@FirstCol) + ' AS ' + QUOTENAME(@FirstCol) + ',' +
        'FORMAT(AVG(' + QUOTENAME(@AvgColumn) + '), ''N2'') AS [Average of ' + @AvgColumn + ']' +
        ' FROM ' +@tableName +
        ' GROUP BY ' + QUOTENAME(@GroupByColumn) +
        ' ORDER BY 2;';

    EXEC sp_executesql @SqlQuery;
END;

Commands completed successfully.

Total execution time: 00:00:00.005

In [27]:
--TO ANSWER QUESTION NMBER 9
CREATE OR ALTER PROC GET_CORR_DEMOavg_CHURN__(@attr VARCHAR(20))
AS
BEGIN
	DECLARE @sql2 NVARCHAR(MAX)
	SET @sql2 = N'
    SELECT AVG(' + QUOTENAME(@attr) + ') AS '+QUOTENAME(@attr)+', s.Customer_Status
    FROM Demographics d
    JOIN Status s ON d.CustomerID = s.CustomerID
	GROUP BY s.Customer_Status';
	EXEC sp_executesql @sql2;
	RETURN
END

Commands completed successfully.

Total execution time: 00:00:00.020

In [50]:
--TO ANSWER QUESTION NMBER 21_1, 21_2
CREATE OR ALTER PROCEDURE GetChurnStatistics
    @GroupByColumn NVARCHAR(100)
AS
BEGIN
    DECLARE @sql NVARCHAR(MAX);

    SET @sql = 
    N'SELECT ' + QUOTENAME(@GroupByColumn) + ' AS ' + QUOTENAME(@GroupByColumn) + ',
       COUNT(*) AS [Total NYMBER of Churned],
       AVG(age) AS [Average of Churned Age]
    FROM 
        status st
    JOIN 
        Demographics demo ON demo.CustomerID = st.CustomerID
    WHERE 
        Customer_Status = ''Churned''
    GROUP BY ' 
        + QUOTENAME(@GroupByColumn) + '
    ORDER BY 
        2 DESC'

    EXEC sp_executesql @sql;
END;


Commands completed successfully.

Total execution time: 00:00:00.002

# **Used Functions**

In [33]:
--To answer Question 3
CREATE OR ALTER FUNCTION GetAverageDownloadByGender()
RETURNS TABLE
AS
RETURN
(
    SELECT
        Gender AS Gender,
        FORMAT(AVG(Avg_Monthly_GB_Download), 'N2') AS [Average Monthly GB Download for all Gender]
    FROM
        Demographics d
    JOIN
        Service s ON d.CustomerID = s.CustomerID
    GROUP BY
        Gender
);

Commands completed successfully.

Total execution time: 00:00:00.011

In [53]:
--To answer question number 13
CREATE OR ALTER FUNCTION HighestStateRateFunc()
RETURNS @T TABLE
(
  Customer_Status NVARCHAR(50),
  Number_of_Customers INT,
  Rate DECIMAL(10, 2)
)
AS
BEGIN
  INSERT INTO @T (Customer_Status, 
                  Number_of_Customers, 
                  Rate)
  SELECT 
     s.Customer_Status,
    count(*) AS [Number oF Customers],
    (CAST(
        COUNT(*) AS DECIMAL(10, 2)) 
        / (SELECT COUNT(*) FROM Status)
        ) * 100 AS [Rate]
  FROM
    Status s
  JOIN
    Demographics d 
    ON s.CustomerID = d.CustomerID
  JOIN
    Location l 
    ON d.Zip_Code = l.Zip_Code
  GROUP BY
    s.Customer_Status;
  ORDER BY
    2 DESC
  RETURN;
END;

: Msg 156, Level 15, State 1, Procedure HighestStateRateFunc, Line 31
Incorrect syntax near the keyword 'ORDER'.

Total execution time: 00:00:00

In [ ]:
-- To answer question Number 16
CREATE OR ALTER FUNCTION dbo.GetAverageDownloadFunction()
RETURNS TABLE
AS
RETURN
(
SELECT DISTINCT
        Internet_Service,
        FORMAT(AVG(Avg_Monthly_GB_Download) OVER (PARTITION BY Internet_Service), 'N2') AS Avg_Monthly_GB_Download
    FROM
        service
);

# **Used Views**

In [36]:
CREATE OR ALTER VIEW city_tenure_consumption(city, state, country, average_tenure, average_consumption)
AS
	SELECT l.city, l.state, l.country, 
           AVG(s.Tenure_in_Months) AS 'Average Tenure in Month', 
           AVG(s.Avg_Monthly_GB_Download) AS 'Average OF Consumption' 
	FROM 
        Location l, Demographics d, Service s
	WHERE 
        l.Zip_Code = d.Zip_Code AND d.CustomerID = s.CustomerID
	GROUP BY 
        l.city, l.state, l.country

GO

Commands completed successfully.

Total execution time: 00:00:00.006

## **Questions & Answers**

### **Q1)Is there a relatuion between Gender and Churned Customers?**

In [21]:
EXEC GET_CORR_DEMO_CHURN__'Gender'

Warning: Null value is eliminated by an aggregate or other SET operation.

(2 rows affected)

Total execution time: 00:00:00.020

Gender,Churned,Stayed
Female,6,10
Male,7,7


### **Q2)  Which Martial Status has the highest Churn rate?**

In [22]:
EXEC GET_CORR_DEMO_CHURN__'Married'

Warning: Null value is eliminated by an aggregate or other SET operation.

(2 rows affected)

Total execution time: 00:00:00.124

Married,Churned,Stayed
No,6,8
Yes,7,9


### **Q2\_1) Is there a relation between Age and Churned?**

In [34]:
EXEC GET_CORR_DEMO_CHURN__'Senior_Citizen'

Warning: Null value is eliminated by an aggregate or other SET operation.

(2 rows affected)

Total execution time: 00:00:00.023

Senior_Citizen,Churned,Stayed
No,9,15
Yes,4,2


### **Q3) Which Gender has more consumption?**

In [35]:
-- Call function GetAverageDownloadByGender
SELECT * FROM GetAverageDownloadByGender();

(2 rows affected)

Total execution time: 00:00:00.017

Gender,Average Monthly GB Download for all Gender
Female,165.00
Male,131.43


**<u>Observation</u>**<span style="white-space-collapse: preserve; color: rgb(255, 255, 255); font-family: Arial, sans-serif; font-size: 14.6667px;">:</span>
**Female** is higher with average in Monthly GB Dawnload  than Males.

### **Q4) Which city has higher tenure and consumption?**

<span style="font-size:10.0pt;font-family:Consolas;mso-fareast-font-family:&quot;Times New Roman&quot;;
mso-bidi-font-family:&quot;Times New Roman&quot;;color:white;mso-themecolor:background1;
mso-font-kerning:0pt;mso-ligatures:none"><br></span>

In [37]:
--Use view named city_tenure_consumption
SELECT 
    TOP(5) city as City, 
    average_tenure [Average Tenure], 
    FORMAT(average_consumption, 'N2') [Average Consumption]
FROM
    city_tenure_consumption
ORDER BY 
    average_tenure DESC, 
    average_consumption DESC

(5 rows affected)

Total execution time: 00:00:00.017

City,Average Tenure,Average Consumption
Paris,21,177.50
Munich,19,170.00
Sydney,18,166.67
Tokyo,18,150.00
Los Angeles,18,150.00


**<u>Observation</u>**<span style="white-space-collapse: preserve; color: rgb(255, 255, 255); font-family: Arial, sans-serif; font-size: 14.6667px;">:</span>  

 From the 5 exists Cities, Paris is the hifhest city with the average of Tenure and Consumption.

### **Q5: Does Customer with High SatisfactionScore have high consumption?**

<span style="color: var(--vscode-foreground);"><br></span>

In [41]:
WITH RankedData AS (
    SELECT 
        st.Satisfaction_Score,
        demo.CustomerID, 
        DENSE_RANK() OVER (ORDER BY ser.Avg_Monthly_GB_Download DESC) AS Gigabyte_Downloads_Dense_Rank,
        ser.Avg_Monthly_GB_Download
    FROM 
        Status st
    JOIN 
        Demographics demo ON demo.CustomerID = st.CustomerID
    JOIN 
        Service ser 
    ON 
        demo.CustomerID = ser.CustomerID
)
SELECT Gigabyte_Downloads_Dense_Rank,
       AVG(RD.Satisfaction_Score) AS[Average of Satisfaction Score]
FROM
       RankedData RD
GROUP BY 
    Gigabyte_Downloads_Dense_Rank
ORDER BY
    2 DESC    


(8 rows affected)

Total execution time: 00:00:00.020

Gigabyte_Downloads_Dense_Rank,Average of Satisfaction Score
8,5
6,4
7,4
1,4
2,3
5,3
4,2
3,1


### **6) What is the average tenure of customers based on their contract type?**

In [4]:
--Call the proc named Average_OF_Column_And_GroupBy
Average_OF_Column_And_GroupBy Contract, Tenure_in_Months, Contract, service


(3 rows affected)

Total execution time: 00:00:00.016

Contract,Average of Tenure_in_Months
Month-to-Month,15.00
One_Year,17.00
Two_Year,21.00


**<u>Observation</u>**<span style="color: rgb(255, 255, 255); font-family: Arial, sans-serif; font-size: 14.6667px; white-space-collapse: preserve;">:</span>

<span style="font-size: 14.6667px; white-space-collapse: preserve;">The max average of time that users use the service is Two years contract.</span>

<span style="font-size: 14.6667px; white-space-collapse: preserve;">The min&nbsp; average of time that users use the service is month to month contract.</span>

### **Q7) How does the average monthly charge vary among different payment methods?**

In [11]:
--Call the proc named Average_OF_Column_And_GroupBy
Average_OF_Column_And_GroupBy Payment_Method, Monthly_Charge, Payment_Method, service

(3 rows affected)

Total execution time: 00:00:00.005

Payment_Method,Average of Monthly_Charge
Credit_Card,55.00
Mailed_Check,77.08
Bank_Withdrawal,81.11


**<u>Observation</u>**<span style="color: rgb(255, 255, 255); font-family: Arial, sans-serif; font-size: 14.6667px; white-space-collapse: preserve;">:</span>

<span style="font-size: 14.6667px; white-space-collapse: preserve;">From the exists data about 30 customers,</span>

<span style="font-size: 14.6667px; white-space-collapse: preserve;">Payment by bank with drawal has the max average in each month,&nbsp; the credit card has the min, and mailes check is moderate.</span>

### **Q8) Which quarter experienced the highest rate for churned customers?**

In [24]:
SELECT 
    --TOP(1)
    serv.Quar AS Quarter, 
    AVG(st.Satisfaction_Score) as 'Average of Satisfaction Score for Churned Customers'
FROM
    Status st
JOIN 
    Service serv
ON 
    st.CustomerID = serv.CustomerID 
WHERE 
    st.Customer_Status = 'Churned' 
GROUP BY 
    serv.Quar
ORDER BY 
    2 DESC

(4 rows affected)

Total execution time: 00:00:00.006

Quarter,Average of Satisfaction Score for Churned Customers
1,3
2,2
4,2
3,1


**<u>Observation</u>**<span style="white-space-collapse: preserve; color: rgb(255, 255, 255); font-family: Arial, sans-serif; font-size: 14.6667px;">:</span>

<span style="font-size: 14.6667px; white-space-collapse: preserve;">Quarter 1 has the max average for the four quarter that the churned customers rate their Satisfaction about the service.</span>

### **Q9) <span style="font-size: 16.38px;">Is there a relationship between the number of dependents (or age) and churn?</span>**

In [26]:
-- Call proc named GET_CORR_DEMOavg_CHURN__
DECLARE @cont_avg TABLE
(
	avg_attr INT,
	Customer_Status VARCHAR(20)
)
INSERT INTO @cont_avg
EXEC GET_CORR_DEMOavg_CHURN__ 'Number_of_Dependents'
SELECT Customer_Status, avg_attr AS [Average Number of Dependents]
FROM @cont_avg


Commands completed successfully.

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.028

Customer_Status,Average Number of Dependents
Churned,4
Stayed,3


### **Q10) What is the distribution of satistifction scores acrros different contract types?**

In [5]:
 SELECT
    sr.Contract,
    COUNT(CASE WHEN st.Satisfaction_Score >= 4 AND st.Satisfaction_Score <= 5 THEN 1 END) AS High_Score_Count,
    COUNT(CASE WHEN st.Satisfaction_Score = 3 THEN 1 END) AS Moderate_Score_Count,
    COUNT(CASE WHEN st.Satisfaction_Score >= 1 AND st.Satisfaction_Score < 3 THEN 1 END) AS Low_Score_Count

FROM
    Service sr
JOIN
    Status st ON sr.CustomerID = st.CustomerID
GROUP BY
    sr.Contract
ORDER BY
    2 DESC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.018

Contract,High_Score_Count,Moderate_Score_Count,Low_Score_Count
Month-to-Month,8,4,1
Two_Year,6,1,2
One_Year,3,0,5


### **Q11)Do customers who stream movies have higher tenure compared to those who don't?**

In [3]:
SELECT
    AVG(CASE WHEN Streaming_Movies = 'Yes' THEN Tenure_in_Months ELSE 0 END) AS Avg_Tenure_Stream_Movies,
    AVG(CASE WHEN Streaming_Movies = 'No' THEN Tenure_in_Months ELSE 0 END) AS Avg_Tenure_No_Stream_Movies
FROM
    Service;

(1 row affected)

Total execution time: 00:00:00.024

Avg_Tenure_Stream_Movies,Avg_Tenure_No_Stream_Movies
7,9


### **Q12) <span style="background-color: rgb(31, 31, 31); color: rgb(204, 204, 204); font-family: Consolas, &quot;Courier New&quot;, monospace; white-space: pre;">Is there a difference in average monthly charges between senior citizens and non-senior citizens?</span>**

In [8]:
SELECT
    FORMAT(AVG(CASE WHEN Senior_Citizen = 'Yes' THEN Monthly_Charge ELSE 0 END), 'N2') AS Avg_Monthly_Charge_Senior,
    FORMAT(AVG(CASE WHEN Senior_Citizen = 'No' THEN Monthly_Charge ELSE 0 END), 'N2') AS Avg_Monthly_Charge_Non_Senior
FROM
    Service
JOIN
    Demographics 
ON
  Service.CustomerID = Demographics.CustomerID;

(1 row affected)

Total execution time: 00:00:00.012

Avg_Monthly_Charge_Senior,Avg_Monthly_Charge_Non_Senior
16.33,55.33


### **Q13\_1) Which Customer status has the highest churn rate?**

In [54]:
SELECT * FROM HighestStateRateFunc();

(2 rows affected)

Total execution time: 00:00:00.022

Customer_Status,Number_of_Customers,Rate
Churned,13,43.33
Stayed,17,56.67


### **Q13\_2) Which Customer state has the highest churn rate?**

In [1]:
SELECT 
     TOP(1) WITH TIES state, 
     COUNT(Customer_Status) [Churn Count]
FROM
     Location l, 
     Demographics d, 
     Status s
WHERE 
     l.Zip_Code=d.Zip_Code AND 
     d.CustomerID=s.CustomerID AND 
     Customer_Status='Churned'
GROUP BY 
     state
ORDER BY 
     2 DESC

(1 row affected)

Total execution time: 00:00:00.059

state,Churn Count
California,5


### **Q14)What is the average tenure for customers who use streaming TV versus those who don't?**

In [1]:
-- Call Proc Average_OF_Column_And_GroupBy
Average_OF_Column_And_GroupBy Streaming_TV, Tenure_in_Months, Streaming_TV, Service
    

(2 rows affected)

Total execution time: 00:00:00.058

Streaming_TV,Average of Tenure_in_Months
No,15.00
Yes,19.00


### **Q15) How does the churn rate vary among different internet service types?** 

<span style="color: var(--vscode-foreground);"><br></span>

In [6]:
SELECT
  S.Internet_Service [Internet Service],
  COUNT(*) AS [Total Customers],
  SUM(CASE WHEN Status.Customer_Status = 'Churned' THEN 1 ELSE 0 END) AS [Churned Customers]
FROM
    Service s
JOIN
  Status ON s.CustomerID = Status.CustomerID
GROUP BY
  S.Internet_Service


(3 rows affected)

Total execution time: 00:00:00.009

Internet Service,Total Customers,Churned Customers
Cable,12,6
DSL,9,3
Fiber Optic,9,4


### **Q16) What is the average monthly download volume for each internet service type?**

<span style="color: var(--vscode-foreground);"><br></span>

In [11]:
SELECT * FROM dbo.GetAverageDownloadFunction();


(3 rows affected)

Total execution time: 00:00:00.012

Internet_Service,Avg_Monthly_GB_Download
Cable,168.33
DSL,108.89
Fiber Optic,164.44


### **Q17) Is there a correlation between satisfaction scores and churn categories?**

<span style="color: var(--vscode-foreground);"><br></span>

In [15]:
DECLARE @s_score INT
SET @s_score = 4

SELECT Churn_Category [Churn Category], 
       COUNT(Satisfaction_Score) [Count Satisfaction]
FROM 
    Status
WHERE 
    Satisfaction_Score = @s_score
GROUP BY 
    Churn_Category
ORDER BY
    2 DESC;    

(4 rows affected)

Total execution time: 00:00:00.006

Churn Category,Count Satisfaction
Price,4
Dissatisfaction,2
Attitude,2
Competitor,1


### **Q18) How does the churn rate change based on the type of offers provided?**

<span style="color: var(--vscode-foreground);"><br></span>

In [55]:
SELECT
    sr.Offer,
    COUNT(sr.Offer) AS [Numer of Churned Use Offer],
    CAST(COUNT(sr.Offer) * 100 / (SELECT COUNT(Customer_Status) 
				FROM Status 
				WHERE Customer_Status = 'Churned') AS INT) 
				AS [Percentage Of Churned]
FROM
    Status st
JOIN
    Service sr 
    ON 
    sr.CustomerID = st.CustomerID
WHERE
    Customer_Status = 'Churned'
GROUP BY
    sr.Offer;

(5 rows affected)

Total execution time: 00:00:00.017

Offer,Numer of Churned Use Offer,Percentage Of Churned
A,5,38
B,1,7
C,1,7
D,4,30
E,2,15


### **Q19) Which payment method is most preferred among customers with high satisfaction scores?**

<span style="color: var(--vscode-foreground);"><br></span>

In [59]:
SELECT
    sr.Payment_Method,
    COUNT(CASE WHEN st.Satisfaction_Score >= 4 AND st.Satisfaction_Score <= 5 THEN 1 END) AS High_Score_Count
FROM
    Service sr
JOIN
    Status st ON sr.CustomerID = st.CustomerID
GROUP BY
    sr.Payment_Method
ORDER BY
    2 DESC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.012

Payment_Method,High_Score_Count
Mailed_Check,7
Credit_Card,6
Bank_Withdrawal,4


### **Q20) Is there a difference in tenure between customers who churned and those who stayed?**

In [60]:
SELECT
    sr.Tenure_in_Months,
    COUNT(CASE WHEN st.Customer_Status = 'Stayed' THEN 1 END) AS Stayed_Customers_Count,
    COUNT(CASE WHEN st.Customer_Status = 'Churned' THEN 1 END) AS Churned_Customers_Count
FROM
    Service sr
JOIN
    Status st ON sr.CustomerID = st.CustomerID
GROUP BY
    sr.Tenure_in_Months
ORDER BY
    sr.Tenure_in_Months;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.003

Tenure_in_Months,Stayed_Customers_Count,Churned_Customers_Count
12,7,3
18,6,6
24,4,4


### **Q21) How does the churn rate & churn Categore differe on age?**

In [52]:
-- Call proc called GetChurnStatistics
GetChurnStatistics Satisfaction_Score

(3 rows affected)

Total execution time: 00:00:00.009

Satisfaction_Score,Total NYMBER of Churned,Average of Churned Age
2,6,40
3,5,38
1,2,54


In [51]:
-- Called proc of GetChurnStatistics
GetChurnStatistics Churn_Category

(5 rows affected)

Total execution time: 00:00:00.015

Churn_Category,Total NYMBER of Churned,Average of Churned Age
Dissatisfaction,4,36
Attitude,3,23
Competitor,3,46
Other,2,48
Price,1,88


### **Q22) What is the average total charge for customers in different cities?**

In [ ]:
SELECT City, 
	FORMAT(AVG(Total_Charges), 'N2') as [Avgerage of Total Charge]
FROM 
     Demographics d,
     location l, 
     Service s
where 
	l.Zip_Code = d.Zip_Code 
    and 
	d.CustomerID = s.CustomerID
group by City
ORDER BY 2

### **Q23) Do customers with a higher number of dependents have higher satisfaction scores?**

In [53]:
SELECT Number_of_Dependents AS [Number of Dependents], 
       AVG(satisfaction_score) as [Satisfaction Score]
FROM Demographics d,
     status s
WHERE
     d.CustomerID = s.CustomerID
GROUP BY Number_of_Dependents
ORDER BY 1 DESC

(8 rows affected)

Total execution time: 00:00:00.006

Number of Dependents,Satisfaction Score
9,4
8,3
7,3
4,2
3,3
2,4
1,3
0,4


There is no relation between number of dependence and <span style="color: rgb(255, 255, 255); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 13px; white-space: pre;">satisfaction score</span>

### **Q24) <span style="font-size: 16.38px;">What is the percentage distribution of churn categories among churned customers?</span>**

In [60]:
SELECT 
    Churn_Category,
    COUNT(Churn_Category) AS Churn_Category_Count,
	FORMAT(100.00 * COUNT(Churn_Category) / SUM(COUNT(Churn_Category)) OVER (), 'N2') AS Percentage
FROM 
	Status
WHERE 
	Customer_Status = 'Churned'
GROUP BY 
	Churn_Category
ORDER BY 
	Churn_Category_Count DESC

(5 rows affected)

Total execution time: 00:00:00.030

Churn_Category,Churn_Category_Count,Percentage
Dissatisfaction,4,30.77
Attitude,3,23.08
Competitor,3,23.08
Other,2,15.38
Price,1,7.69


### Q25)

In [10]:
SELECT
    Streaming_Music,
    AVG(Satisfaction_Score) AS Avg_Satisfaction,    
    COUNT(*) AS Count
FROM
    Status s
JOIN
    Service se ON s.CustomerID = se.CustomerID
GROUP BY
    Streaming_Music;

(2 rows affected)

Total execution time: 00:00:00.019

Streaming_Music,Avg_Satisfaction,Count
No,3,15
Yes,3,15


Audit Table for updates in customer location

In [26]:
--Audit Table for updating customer location
CREATE TABLE loc_history
(
	CustomerID INT,
	UserName VARCHAR(50),
	ModifiedDate DATE,
	Zip_Old INT,
	Zip_New INT
)
GO
CREATE OR ALTER TRIGGER T_loc
ON Demographics
AFTER UPDATE 
AS
	IF UPDATE(Zip_code)
	BEGIN
		DECLARE @old_zip INT , @new_zip INT, @cust_id INT

		DECLARE loc_cursor CURSOR FOR --I used cursor because multiple rows may be updated at the same time
		SELECT Zip_Code, CustomerID FROM deleted

		OPEN loc_cursor
		FETCH NEXT FROM loc_cursor INTO @old_zip, @cust_id
		
		WHILE @@FETCH_STATUS = 0
		BEGIN
			SELECT @new_zip=Zip_Code FROM inserted WHERE CustomerID=@cust_id

			INSERT INTO loc_history VALUES (@cust_id, SUSER_NAME(), GETDATE(), @old_zip, @new_zip)
			
			FETCH NEXT FROM loc_cursor INTO @old_zip, @cust_id
		END

		CLOSE loc_cursor
		DEALLOCATE loc_cursor				
	END


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.015

In [27]:
select * FROM loc_history

(0 rows affected)

Total execution time: 00:00:00.005

CustomerID,UserName,ModifiedDate,Zip_Old,Zip_New


In [31]:
update Demographics
set Zip_Code = 101
where CustomerID = 1

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.031

Announcement
"Location updated successfully, using Zip_Code column by DESKTOP-4H0R5VG\sohila"


In [30]:
SELECT *
FROM loc_history

(1 row affected)

Total execution time: 00:00:00.010

CustomerID,UserName,ModifiedDate,Zip_Old,Zip_New
1,DESKTOP-4H0R5VG\sohila,2023-12-23,100,101
